In [41]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

In [42]:
iris = load_iris()
X = iris.data
y = iris.target
n_classes = len(np.unique(y))

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

AdaBoost from scratch -> it only works for Multi - class

In [44]:
def adaboost_multiclass_scratch(X, y, n_estimators=20):
    n_samples = X.shape[0]
    weights = np.ones(n_samples) / n_samples
    models = []
    alphas = []

    for i in range(n_estimators):
        stump = DecisionTreeClassifier(max_depth=1, random_state=i)
        stump.fit(X, y, sample_weight=weights)
        y_pred = stump.predict(X)

        err = np.sum(weights * (y_pred != y)) / np.sum(weights)
        if err >= 1.0 or err == 0:
            continue

        alpha = np.log((1 - err) / err) + np.log(n_classes - 1)

        weights *= np.exp(alpha * (y_pred != y))
        weights /= np.sum(weights)

        models.append(stump)
        alphas.append(alpha)

    return models, alphas

In [45]:
# Prediction
def predict_multiclass_adaboost(models, alphas, X, n_classes):
    pred = np.zeros((X.shape[0], n_classes))

    for model, alpha in zip(models, alphas):
        pred[np.arange(X.shape[0]), model.predict(X)] += alpha

    return np.argmax(pred, axis=1)

In [46]:
# Train and evaluate scratch AdaBoost
models, alphas = adaboost_multiclass_scratch(X_train, y_train, n_estimators=20)
y_pred_scratch = predict_multiclass_adaboost(models, alphas, X_test, n_classes)

print("Scratch AdaBoost Accuracy:", accuracy_score(y_test, y_pred_scratch))

Scratch AdaBoost Accuracy: 1.0


In [49]:
sk_model = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=20,
    algorithm='SAMME',
    random_state=42
)

sk_model.fit(X_train, y_train)
y_pred_builtin = sk_model.predict(X_test)

print("Built-in AdaBoost Accuracy:", accuracy_score(y_test, y_pred_builtin))

Built-in AdaBoost Accuracy: 1.0


/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(


In [ ]:
sk_model.fit(X_train, y_train)
y_pred_builtin = sk_model.predict(X_test)

print("Built-in AdaBoost Accuracy:", accuracy_score(y_test, y_pred_builtin))